**W pierwszej kolejności usuniemy aktualnie zainstalowaną wersję oprogramowania z NVidia:**


In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi




**Następnie pobierzemy i zainstalujemy oprogramowanie CUDA w wersji 9.2:**


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-11-08 15:20:57--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?uCTw9kemsdzMt1IRkMjTvqv9hSSRvQuB5P6T5IMOw3bCfc6t22E7kFK87BfOYGyMMxGPxiQmTUcjKOErafXhWRgOq574Ulr2uVBHMhXFYfJn_RoVo7jssGREB0fKLyfrO0JOapzltY2O04aul-bl7PzaTlL5nore1omZYV6v3jR42iYBPKs6YJmZyZH3A_IJqPhXopEuIgNcjNLm_Og [following]
--2020-11-08 15:20:57--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?uCTw9kemsdzMt1IRkMjTvqv9hSSRvQuB5P6T5IMOw3bCfc6t22E7kFK87BfOYGyMMxGPxiQmTUcjKOErafXhWRgOq574Ulr2uVBHMhXFYfJn_RoVo7jssGREB0fK

**Sprawdźmy zainstalowaną wersję nvcc:**


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88



**Konieczna jest też instalacja pluginu, który umożliwi nam pisanie kodu w języku C++ w Colab:**


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qbz2_xov
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qbz2_xov
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=4b7b411ee12f74e70e38a2fd5f3e1a6a6a292e95711fe01bd60b78201b407e5e
  Stored in directory: /tmp/pip-ephem-wheel-cache-n2e30529/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


**Teraz możemy wyświetlić informację o dostępnej karcie graficznej. Warto zwrócić uwagę m.in. na liczbę wątków dostępnych w bloku (może się przydać do uzyskania dużej wydajności w zadaniu domowym):**

In [ ]:
%%cu
#include <memory>
#include <iostream>
#include <cuda_runtime.h>
// Main Program
int main(void)
{
    int device_Count = 0;
    cudaGetDeviceCount(&device_Count);
    // This function returns count of number of CUDA enable devices and 0 if there are no CUDA capable devices.
    if (device_Count == 0)
    {
        printf("There are no available device(s) that support CUDA\n");
    }
    else
    {
        printf("Detected %d CUDA Capable device(s)\n", device_Count);
    }
 
    int device = 0;
    int driver_Version, runtime_Version;
 
    cudaDeviceProp device_Property;
    cudaGetDeviceProperties(&device_Property, device);
    printf("\nDevice %d: \"%s\"\n", device, device_Property.name);
    cudaDriverGetVersion(&driver_Version);
    cudaRuntimeGetVersion(&runtime_Version);
    printf(" CUDA Driver Version / Runtime Version %d.%d / %d.%d\n", driver_Version / 1000, (driver_Version % 100) / 10, runtime_Version / 1000, (runtime_Version % 100) / 10);
    printf( " Total amount of global memory: %.0f MBytes (%llu bytes)\n",
    (float)device_Property.totalGlobalMem / 1048576.0f, (unsigned long long) device_Property.totalGlobalMem);
    printf(" (%2d) Multiprocessors", device_Property.multiProcessorCount );
    printf("  GPU Max Clock rate: %.0f MHz (%0.2f GHz)\n", device_Property.clockRate * 1e-3f, device_Property.clockRate * 1e-6f);
    printf("\n");
    printf( " Total amount of global memory: %.0f MBytes (%llu bytes)\n",
    (float)device_Property.totalGlobalMem / 1048576.0f, (unsigned long long) device_Property.totalGlobalMem);
    printf(" Memory Clock rate: %.0f Mhz\n", device_Property.memoryClockRate * 1e-3f);
    printf(" Memory Bus Width: %d-bit\n", device_Property.memoryBusWidth);
    if (device_Property.l2CacheSize)
    {
        printf(" L2 Cache Size: %d bytes\n", device_Property.l2CacheSize);
    }
    printf(" Total amount of constant memory: %lu bytes\n",         device_Property.totalConstMem);
    printf(" Total amount of shared memory per block: %lu bytes\n", device_Property.sharedMemPerBlock);
    printf(" Total number of registers available per block: %d\n", device_Property.regsPerBlock);
    printf("\n");
    printf(" Maximum number of threads per multiprocessor: %d\n",              device_Property.maxThreadsPerMultiProcessor);
    printf(" Maximum number of threads per block: %d\n",         device_Property.maxThreadsPerBlock);
    printf(" Max dimension size of a thread block (x,y,z): (%d, %d, %d)\n",
        device_Property.maxThreadsDim[0],
        device_Property.maxThreadsDim[1],
        device_Property.maxThreadsDim[2]);
    printf(" Max dimension size of a grid size (x,y,z): (%d, %d, %d)\n",
        device_Property.maxGridSize[0],
        device_Property.maxGridSize[1],
        device_Property.maxGridSize[2]);        
}

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
 CUDA Driver Version / Runtime Version 10.1 / 9.2
 Total amount of global memory: 15080 MBytes (15812263936 bytes)
 (40) Multiprocessors  GPU Max Clock rate: 1590 MHz (1.59 GHz)

 Total amount of global memory: 15080 MBytes (15812263936 bytes)
 Memory Clock rate: 5001 Mhz
 Memory Bus Width: 256-bit
 L2 Cache Size: 4194304 bytes
 Total amount of constant memory: 65536 bytes
 Total amount of shared memory per block: 49152 bytes
 Total number of registers available per block: 65536

 Maximum number of threads per multiprocessor: 1024
 Maximum number of threads per block: 1024
 Max dimension size of a thread block (x,y,z): (1024, 1024, 64)
 Max dimension size of a grid size (x,y,z): (2147483647, 65535, 65535)




**Pierwszy przykład pokazuje jak wykonać operacje na karcie graficznej. Napiszemy funkcję, która dodaje dwie liczby i zwraca wynik:**


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
  *c = *a + *b;
}

int main() {
  int a, b, c;
  // host copies of variables a, b & c
  int *d_a, *d_b, *d_c;
  // device copies of variables a, b & c
  int size = sizeof(int);
  // Allocate space for device copies of a, b, c
  cudaMalloc((void **)&d_a, size);
  cudaMalloc((void **)&d_b, size);
  cudaMalloc((void **)&d_c, size);
  // Setup input values  
  c = 0;
  a = 3;
  b = 5;
  // Copy inputs to device
  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
  // Launch add() kernel on GPU
  add<<<1,1>>>(d_a, d_b, d_c);
  // Copy result back to host
  cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
    printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
  printf("result is %d\n",c);
  // Cleanup
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  return 0;
}

result is 8



**W kolejnym przykładzie napiszemy kernel, który wykonuje operację dodawania na całej tablicy i porównamy z szybkością wykonywania tej samej funkcji na CPU:**

In [ ]:
%%cu
#include <iostream>
#include <vector>
#include <random>
#include <chrono>

#define N 10000000

void vector_add(float *out, float *a, float *b, int n) {
    for(int i = 0; i < n; i++){
        out[i] = a[i] + b[i];
    }
}

// Kernel function to add the elements of two arrays
__global__ void add(int n, float *x, float *y, float *out)
{
  for (int i = 0; i < n; i++)
    out[i] = x[i] + y[i];
}

int main(){
    float *a, *b, *out; 
    float *d_a, *d_b, *d_out;
  
    // Allocate memory
    a   = (float*)malloc(sizeof(float) * N);
    b   = (float*)malloc(sizeof(float) * N);
    out = (float*)malloc(sizeof(float) * N);

    // Initialize array
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; b[i] = 2.0f;
    }

    std::chrono::steady_clock::time_point beginCPU = std::chrono::steady_clock::now();
    // add vectors
    vector_add(out, a, b, N);
    std::chrono::steady_clock::time_point endCPU = std::chrono::steady_clock::now();
    std::cout << "Time difference for CPU= " <<
    std::chrono::duration_cast<std::chrono::microseconds>(endCPU - beginCPU).count() << "[µs]\n";

    // print result
    for(int i = 0; i < 10; i++){
        std::cout << out[i] << "\n";
    }

    // GPU CUDA
    // Allocate Unified Memory – accessible from CPU or GPU
    cudaMallocManaged(&d_a, N*sizeof(float));
    cudaMallocManaged(&d_b, N*sizeof(float));
    cudaMallocManaged(&d_out, N*sizeof(float));

    // Initialize array
    for(int i = 0; i < N; i++){
        d_a[i] = 1.0f; d_b[i] = 2.0f;
    }

    std::chrono::steady_clock::time_point beginCUDA = std::chrono::steady_clock::now();
    // add vectors
    add<<<1, 1>>>(N, d_a, d_b, d_out);

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    std::chrono::steady_clock::time_point endCUDA = std::chrono::steady_clock::now();
    std::cout << "Time difference for GPU= " <<
    std::chrono::duration_cast<std::chrono::microseconds>(endCUDA - beginCUDA).count() << "[µs]\n";

    // print result
    for(int i = 0; i < 10; i++){
        std::cout << d_out[i] << "\n";
    }

    // Cleanup
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a);
    free(b);
    free(out);

    std::cout << "Finished\n";
}

Time difference for CPU= 46725[µs]
3
3
3
3
3
3
3
3
3
3
Time difference for GPU= 653822[µs]
3
3
3
3
3
3
3
3
3
3
Finished



**Wywołanie tego samego kodu na GPU jest wielokrotnie wolniejsze niż na CPU. Kluczowe jest tutaj wywołanie kernela "add<<<1, 1>>>". Drugi parametr oznacza liczbę wątków. W tym przypadku kernel wykonany jest w jednym wątku. Procesory na karcie graficznej są wolniejsze niż CPU, do tego dochodzi kopiowanie danych, dlatego w tym przypadku wywołanie kodu na GPU jest wielokrotnie wolniejsze. W kolejnym przykładzie zwiększymy liczbę wątków do 256:**

In [ ]:
%%cu
#include <iostream>
#include <vector>
#include <random>
#include <chrono>

#define N 10000000

void vector_add(float *out, float *a, float *b, int n) {
    for(int i = 0; i < n; i++){
        out[i] = a[i] + b[i];
    }
}

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y, float *out)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  // stride == 256
  // cała tablica dzielona jest na 256 bloków
  for (int i = index; i < n; i += stride)
      out[i] = x[i] + y[i];
}

int main(){
    float *a, *b, *out; 
    float *d_a, *d_b, *d_out;
  
    // Allocate memory
    a   = (float*)malloc(sizeof(float) * N);
    b   = (float*)malloc(sizeof(float) * N);
    out = (float*)malloc(sizeof(float) * N);

    // Initialize array
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; b[i] = 2.0f;
    }

    std::chrono::steady_clock::time_point beginCPU = std::chrono::steady_clock::now();
    // add vectors
    vector_add(out, a, b, N);
    std::chrono::steady_clock::time_point endCPU = std::chrono::steady_clock::now();
    std::cout << "Time difference for CPU= " <<
    std::chrono::duration_cast<std::chrono::microseconds>(endCPU - beginCPU).count() << "[µs]\n";

    // print result
    for(int i = 0; i < 10; i++){
        std::cout << out[i] << "\n";
    }

    // GPU CUDA
    // Allocate Unified Memory – accessible from CPU or GPU
    cudaMallocManaged(&d_a, N*sizeof(float));
    cudaMallocManaged(&d_b, N*sizeof(float));
    cudaMallocManaged(&d_out, N*sizeof(float));

    // Initialize array
    for(int i = 0; i < N; i++){
        d_a[i] = 1.0f; d_b[i] = 2.0f;
    }

    std::chrono::steady_clock::time_point beginCUDA = std::chrono::steady_clock::now();
    // add vectors
    add<<<1, 256>>>(N, d_a, d_b, d_out);

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    std::chrono::steady_clock::time_point endCUDA = std::chrono::steady_clock::now();
    std::cout << "Time difference for GPU= " <<
    std::chrono::duration_cast<std::chrono::microseconds>(endCUDA - beginCUDA).count() << "[µs]\n";

    // print result
    for(int i = 0; i < 10; i++){
        std::cout << d_out[i] << "\n";
    }

    // Cleanup
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a);
    free(b);
    free(out);

    std::cout << "Finished\n";
}

Time difference for CPU= 46972[µs]
3
3
3
3
3
3
3
3
3
3
Time difference for GPU= 31206[µs]
3
3
3
3
3
3
3
3
3
3
Finished



**Dzięki wykorzystaniu 256 wątków (add<<<1, 256>>>) udało się znacząco przyspieszyć obliczenia na GPU. Powinny być nieznacznie szybsze niż obliczenia na CPU. Kluczowa jest również zawartość kernela:**

```
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      out[i] = x[i] + y[i];
```

**Dzięki temu, każy z wątków niezależnie operuje na fragmentach tablicy.** 

Zadanie 1: **Dla lepszego zrozumienia podziału tablicy w kernelu wyświetl wartości 'index', 'stride' i ewentualnie zmienną 'i' za pomocą funkcji 'printf' w kernelu.**

**Karty graficzne z CUDA GPU mają wiele procesorów pogrupowanych w tzw. Streaming Multiprocessors (SMs). Każdy z nich wspiera n wątków. Żeby w pełni wykorzystać możliwości karty graficznej, konieczne jest rozdzielenie zadania na bloki i na wątki wewnątrz bloków <<<liczba_blokow, liczba_watkow>>> (<<<liczba_blokow, rozmiar_bloku>>>):**


In [ ]:
%%cu
#include <iostream>
#include <vector>
#include <random>
#include <chrono>

#define N 10000000

void vector_add(float *out, float *a, float *b, int n) {
    for(int i = 0; i < n; i++){
        out[i] = a[i] + b[i];
    }
}

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y, float *out)
{
    // zwróć uwagę jak obliczany jest index 
    // Czy nie przypomina odwołania do mapy zajętości (tablicy dwuwymiarowej)?
    // Liczbą kolumn w wierszu jest wtedy 'blockDim.x', numerem kolumny 'threadIdx.x',
    // a numerem wiersza jest 'blockIdx.x'.
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
      out[i] = x[i] + y[i];
}

int main(){
    float *a, *b, *out; 
    float *d_a, *d_b, *d_out;
  
    // Allocate memory
    a   = (float*)malloc(sizeof(float) * N);
    b   = (float*)malloc(sizeof(float) * N);
    out = (float*)malloc(sizeof(float) * N);

    // Initialize array
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; b[i] = 2.0f;
    }

    std::chrono::steady_clock::time_point beginCPU = std::chrono::steady_clock::now();
    // add vectors
    vector_add(out, a, b, N);
    std::chrono::steady_clock::time_point endCPU = std::chrono::steady_clock::now();
    std::cout << "Time difference for CPU= " <<
    std::chrono::duration_cast<std::chrono::microseconds>(endCPU - beginCPU).count() << "[µs]\n";

    // print result
    for(int i = 0; i < 10; i++){
        std::cout << out[i] << "\n";
    }

    // GPU CUDA
    // Allocate Unified Memory – accessible from CPU or GPU
    cudaMallocManaged(&d_a, N*sizeof(float));
    cudaMallocManaged(&d_b, N*sizeof(float));
    cudaMallocManaged(&d_out, N*sizeof(float));

    // Initialize array
    for(int i = 0; i < N; i++){
        d_a[i] = 1.0f; d_b[i] = 2.0f;
    }

    std::chrono::steady_clock::time_point beginCUDA = std::chrono::steady_clock::now();
    // add vectors
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    std::cout << "numBlocks " << numBlocks << "\n";
    std::cout << "blockSize " << blockSize << "\n";
    add<<<numBlocks, blockSize>>>(N, d_a, d_b, d_out);

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    std::chrono::steady_clock::time_point endCUDA = std::chrono::steady_clock::now();
    std::cout << "Time difference for GPU= " <<
    std::chrono::duration_cast<std::chrono::microseconds>(endCUDA - beginCUDA).count() << "[µs]\n";

    // print result
    for(int i = 0; i < 10; i++){
        std::cout << d_out[i] << "\n";
    }

    // Cleanup
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a);
    free(b);
    free(out);

    std::cout << "Finished\n";
}

Time difference for CPU= 47678[µs]
3
3
3
3
3
3
3
3
3
3
numBlocks 9766
blockSize 1024
Time difference for GPU= 23775[µs]
3
3
3
3
3
3
3
3
3
3
Finished



**Tym razem udało się uzyskać ponad dwukrotne przyspieszenie.**

**W poprzednich przykładach używaliśmy funkcji cudaMallocManaged() do zarezerwowania pamięci na zmienne przetwarzane przez GPU. Jednocześnie używaliśmy CPU do inicjalizacji tych zmiennych. W tym przypadku używaliśmy tzw. "unified memory", do której ma dostęp CPU i GPU. W tym przypadku drivery do karty graficznej same decydują, gdzie fizycznie umieścić dane. Możliwe jest też używanie funkcji 'cudaMemPrefetchAsync()', która zarezerwuje pamięć na karcie graficznej (nie wszystkie karty to wspierają). W takiej sytaucji inicjalizacja zmiennych przez CPU nie będzie możliwa.**

**W kolejnym przykładzie zmierzymy również czas potrzebny na inicjalizację danych. W przypadku GPU zainicjalizujemy zmienne również na karcie graficznej. Dlatego dodany został kernel do inicjalizacji tych zmiennych na GPU:**

In [ ]:
%%cu
#include <iostream>
#include <vector>
#include <random>
#include <chrono>

#define N 10000000

void vector_add(float *out, float *a, float *b, int n) {
    for(int i = 0; i < n; i++){
        out[i] = a[i] + b[i];
    }
}

// Kernel function to initialize three arrays
__global__
void init(int n, float *x, float *y, float *out)
{
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride){
        x[i] = 1.0;
        y[i] = 2.0;
        out[i] = 0.0;
    }
}

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y, float *out)
{
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride){
      out[i] = x[i] + y[i];
    }
}

int main(){
    float *a, *b, *out; 
    float *d_a, *d_b, *d_out;
  
    // Allocate memory
    a   = (float*)malloc(sizeof(float) * N);
    b   = (float*)malloc(sizeof(float) * N);
    out = (float*)malloc(sizeof(float) * N);

    std::chrono::steady_clock::time_point beginCPU = std::chrono::steady_clock::now();
    // Initialize array
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; b[i] = 2.0f;
    }

    // add vectors
    vector_add(out, a, b, N);
    std::chrono::steady_clock::time_point endCPU = std::chrono::steady_clock::now();
    std::cout << "Time difference for CPU= " <<
    std::chrono::duration_cast<std::chrono::microseconds>(endCPU - beginCPU).count() << "[µs]\n";

    // print result
    for(int i = 0; i < 10; i++){
        std::cout << out[i] << "\n";
    }

    // GPU CUDA
    // Allocate Unified Memory – accessible from CPU or GPU
    cudaMallocManaged(&d_a, N*sizeof(float));
    cudaMallocManaged(&d_b, N*sizeof(float));
    cudaMallocManaged(&d_out, N*sizeof(float));

    // Initialize and add arrays
    std::chrono::steady_clock::time_point beginCUDA = std::chrono::steady_clock::now();
    // add vectors
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    init<<<numBlocks, blockSize>>>(N, d_a, d_b, d_out);
    add<<<numBlocks, blockSize>>>(N, d_a, d_b, d_out);

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    std::chrono::steady_clock::time_point endCUDA = std::chrono::steady_clock::now();
    std::cout << "Time difference for GPU= " <<
    std::chrono::duration_cast<std::chrono::microseconds>(endCUDA - beginCUDA).count() << "[µs]\n";

    // print result
    for(int i = 0; i < 10; i++){
        std::cout << d_out[i] << "\n";
    }

    // Cleanup
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a);
    free(b);
    free(out);

    std::cout << "Finished\n";
}

Time difference for CPU= 105485[µs]
3
3
3
3
3
3
3
3
3
3
Time difference for GPU= 14480[µs]
3
3
3
3
3
3
3
3
3
3
Finished



**Tym razem różnica pomiędzy GPU a CPU jest znacząca na korzyść GPU.**

**Zadanie: zaimplementować prostą sieć neuronową składającą się z neuronów typu LeakyReLU. Wyjście z poprzedniego neuronu jest podawane na wejście neuronu w kolejnej warstwie z wagą równą 1. Parametry neuronów, liczba warstw i liczba neuronów w każdej warstwie powinna być taka sama jak w instrukcji dotyczącej OpenCL. Porównaj implementację CPU i GPU (czas wykonywania i wartości na wyjściu dla 5 pierwszych neuronów).**


**ACKNOWLEDGEMENTS**
- https://stackoverflow.com/questions/51194303/how-to-run-a-python-script-in-a-py-file-from-a-google-colab-notebook
- https://devblogs.nvidia.com/even-easier-introduction-cuda/
- https://devblogs.nvidia.com/unified-memory-cuda-beginners/
